<a href="https://colab.research.google.com/github/hhk919/data_engineering_study/blob/hhk919-assignment-week5%266/data_engineering_study_practice_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASSIGNMENT 1

오늘 내용 정리(5주차)
  * Postgres(raw_data)를 local disk에 tsv형식으로 저장한 다음 Redshift로 한번에 저장하기 위해 AWS S3 bucket에 업로드한다.
  * S3에 접근하기 위해서는 credential이 필요한데 요즘은 iam role(role arn)을 사용한다.
  * airflow는 python기반으로 만들어졌고, web ui는 Flask로 작성되어 있다. 
  * ETL 파일(-.py)에 필요한 variable과 connection은 파일 내부에 하드코딩하는 것보다 airflow의 variable, connection에 변수를 저장해 동적으로 사용하는 것이 좋다.

따라서
<center>
<img src="https://drive.google.com/uc?export=download&id=1R1r7KzGUMM-S3GvIhsAjrhwwogmsG015">
</center>

<br>
connection : Postgres connection 정보, Redshift connection 정보, S3 connection 정보

* airflow의 hook객체로 저장된다.

variable : csv_url, S3 credential에 사용되는 iam role arn

* airflow.models의 Variable객체로 저장된다.








---



ASSIGNMENT2

item_features table, customer_interactions table 추가해보기

- Postgres_to_Redshift.py table 배열에 "item_features"와 "customer_interactions" 추가

In [1]:
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = psycopg2.connect(
    host = "grepp-data.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com",
    port = 5439,
    dbname = "dev",
    user = "hankyoul0919",
    password = "Hankyoul09191!"
)

cursor = conn.cursor()

In [ ]:
sql = "select * from hankyoul0919.item_features limit 10;"
cursor.execute(sql)

col_name = [desc[0] for desc in cursor.description]

print(col_name)
cursor.fetchall()

['item_id', 'quality', 'popularity', 'price']


[(0, 3.1, 'less_popular', 11),
 (1, 4.3, 'new_item', 19),
 (2, 3.8, 'less_popular', 47),
 (3, 3.5, 'less_popular', 13),
 (4, 4.4, 'less_popular', 11),
 (5, 4.7, 'new_item', 34),
 (6, 3.6, 'less_popular', 19),
 (7, 2.0, 'more_popular', 25),
 (8, 3.7, 'new_item', 41),
 (9, 3.9, 'less_popular', 11)]

In [ ]:
sql = "select * from hankyoul0919.customer_interactions limit 10"
cursor.execute(sql)

col_name = [desc[0] for desc in cursor.description]

print(col_name)
cursor.fetchall()

['customer_id', 'datestamp', 'item_id', 'clicked', 'purchased', 'paidamount']


[(0, datetime.date(2016, 4, 11), 16, 1, 0, 0),
 (0, datetime.date(2016, 4, 11), 7, 1, 1, 25),
 (0, datetime.date(2016, 4, 11), 86, 1, 0, 0),
 (0, datetime.date(2016, 4, 14), 22, 0, 0, 0),
 (0, datetime.date(2016, 4, 14), 62, 1, 0, 0),
 (1, datetime.date(2016, 4, 11), 27, 1, 0, 0),
 (10, datetime.date(2016, 4, 12), 41, 0, 0, 0),
 (100, datetime.date(2016, 4, 11), 51, 0, 0, 0),
 (100, datetime.date(2016, 4, 11), 63, 0, 0, 0),
 (100, datetime.date(2016, 4, 11), 85, 0, 0, 0)]



---



ASSIGNMENT3

Summary table을 빌드해보기

예를 들어 customer별로 클릭한 아이템의 수와 구매한 아이템의 수와 총 구매액을 정리해주는 테이블을 만들어볼 수 있다

- build_summary.py에서는 config 디렉토리의 py파일을 읽어 eval함수로 파일 내용을 dict로 변환한다.
- 새로운 summary table을 만들기 위해 목적에 맞는 쿼리문을 작성하면 된다.

<br>

customer별로 클릭한 아이템의 수와 구매한 아이템의 수와 총 구매액을 정리해주는 테이블

→ conversion_summary.py <br>
main sql : <br>
'''select customer_id, sum(clicked) as clicked, sum(purchased) as purchased, sum(paidamount) as paidamount <br> 
from hankyoul0919.customer_interactions <br>
group by 1 <br>
order by 1 '''<br>
<br>

input_check : <br>
sql : 'select count(1) from hankyoul0919.customer_interactions'<br>
count : 100000 <br>
<br>
output_check : <br>
sql : 'select count(1) from {schema}.temp_{table}'<br>
count : 100000 <br>




airflow 실행 결과 :
<center>
<img src='https://docs.google.com/uc?export=download&id=1OMYqAPw4igEFGmW1hH1QGQyA07MJIeIi'
</center>

In [3]:
sql = 'select * from hankyoul0919.user_summary limit 10'
cursor.execute(sql)

col_name = [desc[0] for desc in cursor.description]

print(col_name)
cursor.fetchall()

['channelname', 'sessionid', 'userid', 'ts', 'refunded', 'amount']


[('Naver',
  'c41dd99a69df04044aa4e33ece9c9249',
  184,
  datetime.datetime(2019, 5, 1, 0, 35, 59, 897000),
  None,
  None),
 ('Facebook',
  '0a54b19a13b6712dc04d1b49215423d8',
  251,
  datetime.datetime(2019, 5, 1, 12, 18, 26, 637000),
  None,
  None),
 ('Facebook',
  '05ae14d7ae387b93370d142d82220f1b',
  744,
  datetime.datetime(2019, 5, 1, 14, 17, 54, 83000),
  None,
  None),
 ('Naver',
  '4c4ea5258ef3fb3fb1fc48fee9b4408c',
  265,
  datetime.datetime(2019, 5, 1, 15, 8, 15, 557000),
  None,
  None),
 ('Youtube',
  '60131a2a3f223dc8f4753bcc5771660c',
  45,
  datetime.datetime(2019, 5, 1, 15, 53, 7, 17000),
  None,
  None),
 ('Facebook',
  '90f4760fcc9b69c13da7368c5c2917f3',
  198,
  datetime.datetime(2019, 5, 1, 16, 15, 43, 423000),
  None,
  None),
 ('Instagram',
  '8c97dbeee3b0d40ced7f514b99a93d93',
  702,
  datetime.datetime(2019, 5, 1, 17, 53, 54, 350000),
  None,
  None),
 ('Naver',
  'e10534dd65cf727692c0f9c44ba613f8',
  341,
  datetime.datetime(2019, 5, 1, 18, 32, 18, 390000),


In [4]:
sql = 'select * from hankyoul0919.conversion_summary limit 10'
cursor.execute(sql)

col_name = [desc[0] for desc in cursor.description]

print(col_name)
cursor.fetchall()

['customer_id', 'clicked', 'purchased', 'paidamount']


[(2, 0, 0, 0),
 (4, 0, 0, 0),
 (5, 0, 0, 0),
 (7, 1, 0, 0),
 (9, 0, 0, 0),
 (10, 0, 0, 0),
 (15, 0, 0, 0),
 (16, 1, 0, 0),
 (18, 0, 0, 0),
 (19, 0, 0, 0)]